In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from gensim.models import Word2Vec
from tensorflow.keras.layers import (Bidirectional,
                                     Embedding,
                                     GRU, 
                                     GlobalAveragePooling1D,
                                     GlobalMaxPooling1D,
                                     Concatenate,
                                     SpatialDropout1D,
                                     BatchNormalization,
                                     Dropout,
                                     Dense,
                                     Activation,
                                     concatenate,
                                     Input,
                                     Reshape,
                                     LSTM
                                    )
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical    
import time
import re
import jieba
import json

In [2]:
def cut_words(sentence):
    #print sentence
    return " ".join(jieba.cut(sentence))

In [3]:
# train = pd.read_table('./data/weibo_cuted.txt', header=None,nrows=10000)#encoding='gb2312', \
train_ori = pd.read_csv('./data/nCoV_100k_train.labled.csv',engine ='python')

In [4]:
train_ori.head()
MAX_SEQUENCE_LENGTH = 100
input_categories = '微博中文内容'

In [5]:
# train = train.rename(columns={0:'sentence'}) 
train_ori['sentence'] =  train_ori[input_categories].astype(str).apply(cut_words)


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\2\jieba.cache
Loading model cost 1.314 seconds.
Prefix dict has been built successfully.


In [6]:
train = pd.DataFrame(train_ori,columns=['sentence'])
train.head()
# train.head()

,sentence
0,写 在 年末 冬初 孩子 流感 的 第五天 ， 我们 仍然 没有 忘记 热情 拥抱 这 20...
1,开年 大 模型 … 累到 以为 自己 发烧 了 腰疼 膝盖 疼 腿疼 胳膊 疼 脖子 疼 #...
2,� 偳 癯 空 饩 褪 俏 业 � ， 爹 ， 发烧 快 好 ， 毕竟 美好 的 假期 拿来...
3,新年 的 第一天 感冒 又 发烧 的 也 太衰 了 但是 我要 想着 明天 一定 会 好 的 ?
4,问 ： 我们 意念 里 有 坏 的 想法 了 ， 天神 就 会 给 记下来 ， 那 如果 有...


In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=50000, 
                                                  lower=False,filters="")# 对数据构造词典
tokenizer.fit_on_texts(train['sentence'].values)

json_str = tokenizer.to_json()
json.dump(json_str,open(r'tokenizer_config.json','w'))#文件路径前面加上 r 是为了避免转义
# json_str2= json.load(open('tokenizer_config.json', 'r'))
# tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_str2)

In [8]:
train2num = tokenizer.texts_to_sequences(train['sentence'].values)# 文本 ---> 数字

In [9]:
length = int(np.percentile(list(map(lambda x: len(x),train2num)),95))# 统计95% 字的长度

In [10]:
train2num

[[694,
  14,
  6987,
  327,
  558,
  2,
  8333,
  1,
  42,
  1841,
  62,
  2356,
  6267,
  3485,
  65,
  115,
  78,
  2,
  1487,
  6,
  255,
  114,
  6488,
  8334,
  1,
  3770,
  87,
  327,
  788,
  83,
  3093,
  2,
  8464,
  36211,
  7847,
  1,
  2587,
  205,
  2,
  686,
  1,
  327,
  164,
  529,
  20891,
  6,
  1841,
  69,
  26,
  6488,
  1475,
  1,
  1475,
  12,
  2,
  327,
  313,
  69,
  15808,
  2,
  11602,
  1,
  117,
  13336,
  90,
  168,
  262,
  12,
  2,
  1225,
  1,
  69,
  11603,
  7378,
  16,
  493,
  1,
  33,
  763,
  745,
  8,
  10,
  4511,
  832,
  483,
  25192,
  417,
  12,
  3,
  21,
  22,
  23],
 [6883,
  135,
  9646,
  40,
  19807,
  1037,
  70,
  285,
  8,
  15213,
  7848,
  1861,
  36212,
  6570,
  1861,
  6268,
  1861,
  5,
  36213,
  2,
  5,
  3],
 [56,
  6988,
  10768,
  5573,
  29575,
  27186,
  12206,
  56,
  1,
  4928,
  1,
  285,
  264,
  52,
  1,
  1550,
  1370,
  2,
  1152,
  7849,
  16436,
  7707,
  1,
  1152,
  147,
  49,
  561,
  2646,
  765,
  1,
  492

In [11]:
maxlen = 40
step = 3
sentences = []
labels = []
i = 0
for text in train2num:#[多少个句子[每个句子的数字数组] ]
    # 根据空格划分为数组,取40个词作为样本, 以及其后的1个词作为标签
    for i in range(0, len(text) - maxlen, step):
        # 除了空格40个词
        sentences.append(text[i: i + maxlen])
        labels.append(text[i + maxlen])

In [12]:
sentences[0]

[694,
 14,
 6987,
 327,
 558,
 2,
 8333,
 1,
 42,
 1841,
 62,
 2356,
 6267,
 3485,
 65,
 115,
 78,
 2,
 1487,
 6,
 255,
 114,
 6488,
 8334,
 1,
 3770,
 87,
 327,
 788,
 83,
 3093,
 2,
 8464,
 36211,
 7847,
 1,
 2587,
 205,
 2,
 686]

In [13]:
train_last = tf.keras.preprocessing.sequence.pad_sequences(sentences, maxlen=maxlen,
                                                      padding='pre',truncating='pre',value=0.0)

In [14]:
word_vocab = tokenizer.word_index
print(len(word_vocab))

154228


In [15]:
word_vocab

{'，': 1,
 '的': 2,
 '?': 3,
 '/': 4,
 '#': 5,
 '。': 6,
 '@': 7,
 '了': 8,
 ':': 9,
 '！': 10,
 '是': 11,
 '我': 12,
 '、': 13,
 '在': 14,
 '疫情': 15,
 '：': 16,
 '】': 17,
 '武汉': 18,
 '【': 19,
 '肺炎': 20,
 '展开': 21,
 '全文': 22,
 'c': 23,
 '都': 24,
 '？': 25,
 '有': 26,
 '和': 27,
 '人': 28,
 '不': 29,
 '新型': 30,
 '也': 31,
 '“': 32,
 '你': 33,
 '”': 34,
 '2': 35,
 '就': 36,
 '月': 37,
 '日': 38,
 '冠状病毒': 39,
 '…': 40,
 '1': 41,
 '我们': 42,
 '加油': 43,
 '为': 44,
 '##': 45,
 'O': 46,
 '感染': 47,
 '口罩': 48,
 '要': 49,
 '确诊': 50,
 '医院': 51,
 '好': 52,
 '病例': 53,
 '中国': 54,
 '病毒': 55,
 '�': 56,
 '说': 57,
 '患者': 58,
 '（': 59,
 '到': 60,
 '被': 61,
 '没有': 62,
 '-': 63,
 '防控': 64,
 '这': 65,
 '）': 66,
 '例': 67,
 '还': 68,
 '会': 69,
 '自己': 70,
 '一个': 71,
 '去': 72,
 '对': 73,
 '微博': 74,
 '能': 75,
 '视频': 76,
 '他': 77,
 '年': 78,
 '可以': 79,
 '这个': 80,
 '等': 81,
 '很': 82,
 '上': 83,
 '啊': 84,
 '新冠': 85,
 '让': 86,
 '给': 87,
 '工作': 88,
 '链接': 89,
 '多': 90,
 '大家': 91,
 '吧': 92,
 '一起': 93,
 '来': 94,
 '中': 95,
 '网页': 96,
 '吃': 97,
 '看':

In [16]:
from gensim.models import Word2Vec

file_name = './word2vec/Word2Vec_word_200.model'
if not os.path.exists(file_name):
    print('training')
    model_word2vec = Word2Vec([document.split(' ')for document in train['sentence'].values],
                     size=200, 
                     window=5,
                     iter=10, 
                     workers=11, 
                     seed=2018, 
                     min_count=2)

    model_word2vec.save(file_name)
else:
    print('loading')
    model_word2vec = Word2Vec.load(file_name)
print("add word2vec finished....")    

loading


C:\ProgramData\Anaconda3\envs\jupyter\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


add word2vec finished....


In [17]:
len(word_vocab)

154228

In [18]:
## 2、word embedding构建

In [19]:
count = 0
word_vocal_reverse = {}
word_vocal_reverse[0] = 'SPACE'
embedding_matrix = np.zeros((len(word_vocab)+1, 200)) # 为什么加1 ? 因为字典没有0, 我们为补全造了一个0
for word, i in word_vocab.items():
    word_vocal_reverse[i] = word
    embedding_vector = model_word2vec.wv[word] if word in model_word2vec else None # 取出向量
    if embedding_vector is not None:
        count += 1
        embedding_matrix[i] = embedding_vector# i 表示词典中的位置, 通过embedding,来代替 字的向量表示
    else:
        unk_vec = np.random.random(200) * 0.5
        unk_vec = unk_vec - unk_vec.mean()# 减去均值
        embedding_matrix[i] = unk_vec

C:\ProgramData\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys


In [20]:
np.save('idsMatrix', embedding_matrix)

embedding_matrix,word_vocal_reverse

(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.46237981, -0.00398681, -0.6069175 , ...,  0.83942533,
         -1.43061078, -0.80574322],
        [-0.19985735,  1.69759762,  0.96107876, ...,  0.86869043,
         -0.8077023 ,  0.10116135],
        ...,
        [ 0.19661183,  0.04551218,  0.15216448, ...,  0.13981429,
         -0.14614897,  0.00410833],
        [ 0.18069148,  0.02912275,  0.05407618, ..., -0.1678914 ,
          0.09074305,  0.17332692],
        [-0.14575811, -0.03374856,  0.12297662, ..., -0.02900978,
         -0.11275345,  0.21407788]]),
 {0: 'SPACE',
  1: '，',
  2: '的',
  3: '?',
  4: '/',
  5: '#',
  6: '。',
  7: '@',
  8: '了',
  9: ':',
  10: '！',
  11: '是',
  12: '我',
  13: '、',
  14: '在',
  15: '疫情',
  16: '：',
  17: '】',
  18: '武汉',
  19: '【',
  20: '肺炎',
  21: '展开',
  22: '全文',
  23: 'c',
  24: '都',
  25: '？',
  26: '有',
  27: '和',
  28: '人',
  29: '不',
  30: '新型',
  31: '也',
  32: '“',
  33: '你',

In [21]:
# labels = np.zeros((len(sentences), embedding_matrix.shape[0]), dtype=np.bool)
# i = 0;
# for charnum in labels_temp:
#     labels[i, charnum] = 1
#     i+=1
# labels = labels_temp

In [22]:
labels[5]

327

In [23]:
#     output = Dense(class_num)(x)# 全连接层
#     last_num = sent_length*class_num
#     output = Reshape((last_num,))(output)
def build_model(sent_length, embeddings_weight):
    content = Input(shape=(sent_length,), dtype='int32')

    embedding = Embedding(
        name="word_embedding",
        input_dim=embeddings_weight.shape[0],
        weights=[embeddings_weight],
        output_dim=embeddings_weight.shape[1],
        trainable=False)
    x = embedding(content)

#     x = LSTM(128,input_shape=(sent_length,class_num))(x)
    x = LSTM(128)(x)
    x = Dense(embedding_matrix.shape[0])(x)
    output = Activation(activation="softmax")(x)    
    
#     x = SpatialDropout1D(0.2)(x)

#     x = Bidirectional(GRU(200, return_sequences=True))(x)
#     x = Bidirectional(GRU(200, return_sequences=True))(x)

#     avg_pool = GlobalAveragePooling1D()(x)# 全军平均池化
#     max_pool = GlobalMaxPooling1D()(x)# 全局最大池化

#     conc = concatenate([avg_pool, max_pool]) # 特征放到一起，
#     output = Dense(1)(x)# 全连接层
#     output = BatchNormalization()(x)


#     x = BatchNormalization()(x)
#     x = Activation(activation="relu")(x)
#     x = Dropout(0.2)(x)
#     x = Dense(500)(x)
#     x = BatchNormalization()(x)
#     x = Activation(activation="relu")(x)
#     output = Dense(19, activation="softmax")(x)

    model = tf.keras.models.Model(inputs=content, outputs=output)
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    return model

In [24]:
model = build_model(maxlen, embedding_matrix)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 40)]              0         
_________________________________________________________________
word_embedding (Embedding)   (None, 40, 200)           30845800  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               168448    
_________________________________________________________________
dense (Dense)                (None, 154229)            19895541  
_________________________________________________________________
activation (Activation)      (None, 154229)            0         
Total params: 50,909,789
Trainable params: 20,063,989
Non-trainable params: 30,845,800
_________________________________________________________________


In [26]:
## 模型训练

In [27]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard
# 检查点保存至的目录
checkpoint_dir = './checkpoints_reply'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
print(checkpoint_dir)
print(checkpoint_prefix)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=6)

plateau = ReduceLROnPlateau(monitor="val_accuracy", 
                            verbose=1, 
                            mode='max', 
                            factor=0.5, 
                            patience=3)
checkpoint = ModelCheckpoint(checkpoint_prefix, 
                             monitor='val_accuracy',
                             verbose=2, 
                             save_best_only=True,
                             mode='max',
                             save_weights_only=True)

./checkpoints_reply
./checkpoints_reply\ckpt_{epoch}


In [28]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_last,labels,
                                               test_size=0.1,random_state=666)# 0.1 或 0.05 ,因为数据量比较大，

In [29]:
X_train.shape,X_val.shape,y_train,y_val

((588756, 40),
 (65418, 40),
 [43643,
  48,
  57,
  2105,
  1,
  5,
  7771,
  6959,
  6,
  25,
  1319,
  2153,
  25418,
  62,
  78,
  890,
  9,
  4,
  549,
  90,
  6,
  3123,
  3494,
  1282,
  49480,
  10,
  23,
  13,
  6,
  922,
  13100,
  363,
  77,
  18940,
  1084,
  113,
  742,
  204,
  10,
  73,
  35155,
  427,
  124,
  664,
  25,
  24,
  20,
  47,
  4970,
  1353,
  142,
  8100,
  1,
  4,
  20,
  140,
  44,
  1908,
  2513,
  23,
  11018,
  3363,
  32,
  48,
  27,
  96,
  839,
  1,
  1588,
  2429,
  7,
  39,
  18635,
  445,
  545,
  1,
  18,
  326,
  9039,
  3,
  135,
  184,
  2,
  19792,
  238,
  11,
  4188,
  7753,
  8745,
  1508,
  13,
  1,
  3444,
  2,
  139,
  3136,
  30191,
  87,
  124,
  216,
  184,
  2,
  6731,
  118,
  413,
  26,
  10016,
  11,
  269,
  1,
  30,
  83,
  90,
  2634,
  21,
  5184,
  67,
  106,
  320,
  147,
  330,
  1563,
  9,
  37,
  8,
  34312,
  1024,
  9411,
  3482,
  49511,
  514,
  1,
  3,
  1,
  1,
  78,
  81,
  1,
  51,
  1,
  931,
  809,
  1482,
  6

In [30]:
def preprocess(inputs,labels):
    #最简单的预处理函数:	转numpy为Tensor、分类问题需要处理label为one_hot编码、处理训练数据	
    #把numpy数据转为Tensor
    labels = tf.cast(labels, dtype=tf.int32)
    #labels 转为one_hot编码
    labels = tf.one_hot(labels, depth=embedding_matrix.shape[0])
    return inputs,labels

In [31]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(10000).map(preprocess).batch(64)# 打乱,随机取batch训练
val_ds = tf.data.Dataset.from_tensor_slices((X_val,y_val)).map(preprocess).batch(64)# 测试集批处理的大小可以和训练机匹配的大小不一样?

In [32]:
train_ds,val_ds

(<BatchDataset shapes: ((None, 40), (None, 154229)), types: (tf.int32, tf.float32)>,
 <BatchDataset shapes: ((None, 40), (None, 154229)), types: (tf.int32, tf.float32)>)

In [33]:
model.fit(train_ds,
          epochs=15,
          validation_data=val_ds,
          callbacks=[early_stopping, plateau, checkpoint],
          verbose=1)

Train for 9200 steps, validate for 1023 steps
Epoch 1/15
9199/9200 [============================>.] - ETA: 0s - loss: 6.2424 - accuracy: 0.1926
Epoch 00001: val_accuracy improved from -inf to 0.23382, saving model to ./checkpoints_reply\ckpt_1
9200/9200 [==============================] - 2816s 306ms/step - loss: 6.2424 - accuracy: 0.1926 - val_loss: 5.5946 - val_accuracy: 0.2338
Epoch 2/15
9199/9200 [============================>.] - ETA: 0s - loss: 5.0102 - accuracy: 0.2517
Epoch 00002: val_accuracy improved from 0.23382 to 0.26141, saving model to ./checkpoints_reply\ckpt_2
9200/9200 [==============================] - 2796s 304ms/step - loss: 5.0102 - accuracy: 0.2517 - val_loss: 5.3291 - val_accuracy: 0.2614
Epoch 3/15
9199/9200 [============================>.] - ETA: 0s - loss: 4.5337 - accuracy: 0.2792
Epoch 00003: val_accuracy improved from 0.26141 to 0.27780, saving model to ./checkpoints_reply\ckpt_3
9200/9200 [==============================] - 2766s 301ms/step - loss: 4.5336 -

In [34]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [35]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [36]:
train_last.shape,train_last[0]

((654174, 40),
 array([  694,    14,  6987,   327,   558,     2,  8333,     1,    42,
         1841,    62,  2356,  6267,  3485,    65,   115,    78,     2,
         1487,     6,   255,   114,  6488,  8334,     1,  3770,    87,
          327,   788,    83,  3093,     2,  8464, 36211,  7847,     1,
         2587,   205,     2,   686]))

In [37]:
import random,sys
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print('----- Generating text after Epoch: %d' % epoch)
    text = []
    while len(text) < 40:        
        random_array_index =  random.randint(0, len(train2num) - 1)
        print(random_array_index)
        text = train2num[random_array_index]
    print(text)
    start_index = random.randint(0, len(text) - maxlen - 1)
    print(start_index)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
    for diversity in [0.2, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        for word in sentence:
            generated += word_vocal_reverse[word]        
        print('----- Generating with seed:')
        sys.stdout.write(generated)
        lalala = ''
        for i in range(20):#400
            x_pred = np.zeros((1, maxlen))
            for t, char in enumerate(sentence):
                #print(t,char)
                #x_pred[0, t, char_indices[char]] = 1.
                x_pred[0, t] = char
                
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = word_vocal_reverse[next_index]
            generated += next_char
            sentence = sentence[1:] 
            sentence.append(next_index)
            print(generated)
#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print('end')

In [38]:
on_epoch_end(1,'2')



----- Generating text after Epoch: 1
96540
[5, 833, 2846, 5, 115, 1414, 7788, 5, 1021, 7788, 5, 1308, 591, 3, 3, 10575, 5, 35315, 1021, 7788, 5, 56, 3395, 4278, 48135, 111, 788, 8, 24291, 180, 1487, 1, 7, 56, 6427, 48607, 27665, 56, 1489, 380, 230, 2, 70, 16, 148, 153, 125, 71, 806, 27610, 205, 533, 43, 10, 3]
7
----- diversity: 0.2
----- Generating with seed:
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量，
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将

C:\ProgramData\Anaconda3\envs\jupyter\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量，我们一起加油！?展开全文c
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量，我们一起加油！?展开全文c?
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量，我们一起加油！?展开全文c??
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做吧，再坚持，再坚持力量，我们一起加油！?展开全文c???
----- diversity: 1.0
----- Generating with seed:
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家，
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家，将
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家，将全力支持
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家，将全力支持湖北
#电影宠爱#暖哭??超棒#吴磊电影宠爱#�偽吴磊今天穿了功过第一天，@�偣Ρ匮�即将17岁的自己：不要再做事实，请大家，将全力支持湖北城里
#电影宠爱#暖哭??超棒#吴磊电影宠